In [1]:
#Import package pandas for data analysis
import pandas as pd
# Import package numpy for numeric computing
import numpy as np
import seaborn as sns
# Import package matplotlib for visualisation/plotting
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

from patsy import dmatrices
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score

import pymysql

import matplotlib.pyplot as plt

#For showing plots directly in the notebook run the command below
%matplotlib inline

# For saving multiple plots into a single pdf file
from matplotlib.backends.backend_pdf import PdfPages

import scipy.stats as ss

# ignore warnings
import warnings
warnings.filterwarnings('ignore')#Import package pandas for data analysis
import pandas as pd
# Import package numpy for numeric computing
import numpy as np
import seaborn as sns
# Import package matplotlib for visualisation/plotting
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

from patsy import dmatrices
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score

import pymysql

import matplotlib.pyplot as plt

#For showing plots directly in the notebook run the command below
%matplotlib inline

# For saving multiple plots into a single pdf file
from matplotlib.backends.backend_pdf import PdfPages

import scipy.stats as ss

# ignore warnings
import warnings
warnings.filterwarnings('ignore')#Import package pandas for data analysis
import pandas as pd
# Import package numpy for numeric computing
import numpy as np
import seaborn as sns
# Import package matplotlib for visualisation/plotting
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

from patsy import dmatrices
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score

import pymysql

import matplotlib.pyplot as plt

#For showing plots directly in the notebook run the command below
%matplotlib inline

# For saving multiple plots into a single pdf file
from matplotlib.backends.backend_pdf import PdfPages

import scipy.stats as ss

# ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [16]:
#read in csv prepared from previous data cleaning 
df = pd.read_csv('feat_df.csv')

In [17]:
df.head(50)

,Unnamed: 0,TRIPID,PROGRNUMBER,DIRECTION,ACTUALTIME_ARR,hour,dayofweek,trip_dep_time,weekday,rain,temp,humidity,wind_speed,wind_dir,sun,visibility,cloud_height,cloud_cover,holiday
0,0,5956265,2,1,36075,10,0,0,0,0.0,5.3,7,17,240,0.9,30000,200,4,1
1,1,5956265,3,1,36091,10,0,0,0,0.0,5.3,7,17,240,0.9,30000,200,4,1
2,2,5956265,4,1,36125,10,0,0,0,0.0,5.3,7,17,240,0.9,30000,200,4,1
3,3,5956265,5,1,36183,10,0,0,0,0.0,5.3,7,17,240,0.9,30000,200,4,1
4,4,5956265,6,1,36254,10,0,0,0,0.0,5.3,7,17,240,0.9,30000,200,4,1
5,5,5956265,7,1,36315,10,0,0,0,0.0,5.3,7,17,240,0.9,30000,200,4,1
6,6,5956265,8,1,36361,10,0,0,0,0.0,5.3,7,17,240,0.9,30000,200,4,1
7,7,5956265,9,1,36438,10,0,0,0,0.0,5.3,7,17,240,0.9,30000,200,4,1
8,8,5956265,10,1,36480,10,0,0,0,0.0,5.3,7,17,240,0.9,30000,200,4,1
9,9,5956265,11,1,36500,10,0,0,0,0.0,5.3,7,17,240,0.9,30000,200,4,1


In [18]:
#remove first trip - no departure time available for it
df = df.iloc[41:]


In [19]:
#reset index
df = df.reset_index()

In [20]:
df.head()

,index,Unnamed: 0,TRIPID,PROGRNUMBER,DIRECTION,ACTUALTIME_ARR,hour,dayofweek,trip_dep_time,weekday,rain,temp,humidity,wind_speed,wind_dir,sun,visibility,cloud_height,cloud_cover,holiday
0,41,41,5956266,1,2,39636,11,0,39636,0,0.0,6.3,7,22,250,0.5,30000,200,4,1
1,42,42,5956266,2,2,39675,11,0,39636,0,0.0,6.3,7,22,250,0.5,30000,200,4,1
2,43,43,5956266,3,2,39744,11,0,39636,0,0.0,6.3,7,22,250,0.5,30000,200,4,1
3,44,44,5956266,4,2,39782,11,0,39636,0,0.0,6.3,7,22,250,0.5,30000,200,4,1
4,45,45,5956266,5,2,39809,11,0,39636,0,0.0,6.3,7,22,250,0.5,30000,200,4,1


In [21]:
df = df.drop(['Unnamed: 0', 'index'], axis = 1)

In [22]:
df.head()

,TRIPID,PROGRNUMBER,DIRECTION,ACTUALTIME_ARR,hour,dayofweek,trip_dep_time,weekday,rain,temp,humidity,wind_speed,wind_dir,sun,visibility,cloud_height,cloud_cover,holiday
0,5956266,1,2,39636,11,0,39636,0,0.0,6.3,7,22,250,0.5,30000,200,4,1
1,5956266,2,2,39675,11,0,39636,0,0.0,6.3,7,22,250,0.5,30000,200,4,1
2,5956266,3,2,39744,11,0,39636,0,0.0,6.3,7,22,250,0.5,30000,200,4,1
3,5956266,4,2,39782,11,0,39636,0,0.0,6.3,7,22,250,0.5,30000,200,4,1
4,5956266,5,2,39809,11,0,39636,0,0.0,6.3,7,22,250,0.5,30000,200,4,1


In [23]:
#write over previous csv with this better one
df.to_csv('feat_df.csv')